In [284]:
import dlib
import cv2
import os
import pandas as pd
import numpy as np

In [285]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [286]:
def convertToGray(img): # function that convert image to grayscale.
	return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [287]:
def get_landmarks(imag):
	landmarks = []
	detections = detector(imag, 1)
    
	for k,d in enumerate(detections): #For all detected face instances individually
		shape = predictor(imag, d) #Draw Facial Landmarks with the predictor class
		xlist = []
		ylist = []
		for i in range(1,68): #Store X and Y coordinates in two lists
			xlist.append(float(shape.part(i).x))
			ylist.append(float(shape.part(i).y))

		for x, y in zip(xlist, ylist): #Store all landmarks in one list in the format x1,y1,x2,y2,etc.
			landmarks.append(x)
			landmarks.append(y)
            
	if len(detections) > 0:
		return landmarks
	else: #If no faces are detected, return error message to other function to handle
		landmarks = "error"
		return landmarks

In [288]:
def extractFrames(pathIn, pathOut):
	df = pd.DataFrame()
	os.mkdir(pathOut)

	cap = cv2.VideoCapture(pathIn)
	count = 0
	bias = 50
	while (cap.isOpened() and count<10): # o contador<10 serve para tirar apenas os 10 primeiros frames do video

		# Capture frame-by-frame
		ret, frame = cap.read()

		if ret == True:
			print('Read %d frame: ' % count, ret)

			frame = convertToGray(frame) # convert image to gray
			# uses the classifier HaarCascade to extract the faces.
			haar_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
			faces = haar_face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5) # detecta quantas faces ha na imagem

			for (x,y,w,h) in faces: # Para cada
				crop_img = frame[y-bias: y+h+bias, x-bias: x+w+bias]

			landmark = get_landmarks(crop_img) # Aqui recebo lista com landmarks.
			#print(type(landmark)) # Podemos ver com isso que a variavel retornada pela funcao get_landmarks eh do tipo list.
			#print(len(landmark)) # Tamanho da lista
			array = np.array([landmark])
			land_array = pd.DataFrame(array)
			df = df.append(land_array)
			#print(df)

			clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
			clahe_image = clahe.apply(crop_img) # Aplicação do Clahe a imagem ja em cinza
			detections = detector(clahe_image, 1)#Detect the faces in the image
			for k,d in enumerate(detections): #For each detected face
            			shape = predictor(clahe_image, d) #Get coordinates
            			for i in range(1,68): #There are 68 landmark points on each face
                     			#For each point, draw a red circle with thickness2 on the original frame
                    			cv2.circle(crop_img, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2)
                                # Colocar as coordenadas em um DataFrame

			cv2.imwrite(os.path.join(pathOut, "frame{:d}.jpg".format(count)), crop_img)  # save crop_img as JPEG file

			count += 1
		else:
			break

	print(df)
	# When everything done, release the capture
	cap.release()
	#cv2.destroyAllWindows()


In [289]:
def createDataFrame():

	df = pd.DataFrame()
	print(df)

In [290]:
def main():
	extractFrames('Em1_Fala1_CarolinaHolly.mp4', 'A10')

In [291]:
if __name__=="__main__":
	main()

Read 0 frame:  True
Read 1 frame:  True
Read 2 frame:  True
Read 3 frame:  True
Read 4 frame:  True
Read 5 frame:  True
Read 6 frame:  True
Read 7 frame:  True
Read 8 frame:  True
Read 9 frame:  True
    0      1      2      3      4      5      6      7      8      9    ...  \
0  88.0  263.0   96.0  312.0  103.0  360.0  115.0  408.0  138.0  450.0  ...   
0  88.0  262.0   96.0  311.0  102.0  360.0  115.0  407.0  138.0  450.0  ...   
0  91.0  267.0  100.0  316.0  106.0  365.0  119.0  413.0  143.0  456.0  ...   
0  93.0  267.0  102.0  316.0  109.0  366.0  121.0  414.0  144.0  457.0  ...   
0  90.0  264.0   98.0  313.0  106.0  362.0  118.0  410.0  142.0  453.0  ...   
0  87.0  261.0   96.0  310.0  103.0  359.0  115.0  407.0  138.0  451.0  ...   
0  89.0  269.0   97.0  319.0  104.0  368.0  117.0  416.0  140.0  460.0  ...   
0  87.0  265.0   95.0  315.0  102.0  365.0  115.0  414.0  138.0  459.0  ...   
0  86.0  265.0   95.0  315.0  102.0  365.0  115.0  413.0  138.0  458.0  ...   
0  85.0  2